<a href="https://colab.research.google.com/github/Lipeya/EMNIST_classify/blob/main/RESNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# train 데이터 처리

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Conv1D, MaxPooling2D, MaxPooling1D, Dropout, Flatten
from keras import losses
from keras import optimizers
from keras.utils import np_utils
import numpy as np
import pandas as pd
import csv
import math

In [ ]:
#google drive 연동
from google.colab import drive
drive.mount('/content/mnt')

In [ ]:
class_array = ['a', 'b','c', 'd', 'e', 'f', 'g', 'h', 'i','j','k','l','m','n','o','p', 'q', 'r','s', 't','u','v','w','x','y','z']
class_dict = {'a':0, 'b':1,'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7, 'i':8,'j':9,'k':10,'l':11,'m':12,'n':13,'o':14,'p':15, 'q':16, 'r':17,'s':18, 't':19,'u':20,'v':21,'w':22,'x':23,'y':24,'z':25}
class_size =len(class_array)
print(class_size)

In [ ]:
import pandas as pd

train_data_list = pd.read_csv("/content/mnt/MyDrive/project/emnist-letters-train.csv") #,chunksize = 100000)
test_data_list = pd.read_csv("/content/mnt/MyDrive/project/emnist-letters-test.csv")
predict_data_list = pd.read_csv("/content/mnt/MyDrive/project/predict_char.csv")
predict_train = pd.read_csv("/content/mnt/MyDrive/project/encodedpdc.csv")
predict_test = pd.read_csv("/content/mnt/MyDrive/project/encodedpdctest.csv")


In [ ]:
train_data =  np.array(train_data_list.iloc[:,1:])
train_data = train_data.reshape(train_data.shape[0],28,28,1)
train_data_class = np.array(train_data_list.iloc[:,0])
train_data_class = train_data_class - 1
origin_tdc = train_data_class
train_data_class = np_utils.to_categorical(train_data_class ,26)

In [ ]:
#LSTMX
import tensorflow as tf
input = tf.keras.Input(shape=(28,28,1))
efnet = tf.keras.applications.ResNet50(weights=None,
                                             include_top = False, 
                                             input_tensor = input)
# Now that we apply global max pooling.
gap = tf.keras.layers.GlobalMaxPooling2D()(efnet.output)
gap = Dropout(0.2)(gap)
gap = Dense(52, activation = 'relu')(gap)
gap = Dropout(0.2)(gap)
# Finally, we add a classification layer.
output = tf.keras.layers.Dense(class_size, activation='softmax')(gap)

# bind all
func_model = tf.keras.Model(efnet.input, output)
func_model.compile(
          loss  = tf.keras.losses.CategoricalCrossentropy(),
          metrics = tf.keras.metrics.CategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())

In [ ]:
#LSTMO
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, TimeDistributed, Dropout, concatenate, Bidirectional, LSTM, Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras import Model
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

char_inputs = Input(shape=(class_size,),dtype='float', name='char_predicts_input')
predict = Dense(52, activation = 'relu')(char_inputs)
predict = Dropout(0.2)(predict)
input = tf.keras.Input(shape=(28,28,1))
efnet = tf.keras.applications.ResNet50(weights=None,
                                             include_top = False, 
                                             input_tensor = input)
# Now that we apply global max pooling.
gap = tf.keras.layers.GlobalMaxPooling2D()(efnet.output)

# Finally, we add a classification layer.
#gap = tf.keras.layers.Dense(3136, activation='relu', use_bias=True)(gap)


gap = Dense(52, activation = 'relu')(gap)
gap = Dropout(0.2)(gap)

output = concatenate([gap, predict])
output = Dense(52, activation = 'relu')(output)
output = Dropout(0.2)(output)
output = Dense(26,activation = 'softmax')(output)
# bind all
func_model = tf.keras.Model(inputs=[input,char_inputs], outputs=[output])
func_model.compile(
          loss  = tf.keras.losses.CategoricalCrossentropy(),
          metrics = tf.keras.metrics.CategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())

In [ ]:
func_model.summary()

In [ ]:
pdc = np.array(predict_train.iloc[:,1:])

# 학습

In [ ]:
# fit 
func_model.fit(train_data, train_data_class, batch_size=128, epochs=5, validation_split = 0.2, verbose = 1)

# 결과 확인

In [ ]:
#test 처리
test_data =  np.array(test_data_list.iloc[:,1:])
test_data = test_data.reshape(test_data.shape[0],28,28,1)
test_data_class = np.array(test_data_list.iloc[:,0]) -1
origin_test = test_data_class
test_data_class = np_utils.to_categorical(test_data_class, class_size)

print(test_data_class)

In [ ]:
pdctest = np.array(predict_test.iloc[:,1:])

In [ ]:
#LSTM 모델 실험
score = func_model.evaluate(x=[test_data,pdctest],y= test_data_class, verbose=0)

print("loss : " + str(score[0]))
print("accuracy : " + str(score[1]))

In [ ]:
#LSTM 모델 X 실험
score = func_model.evaluate(x=test_data,y= test_data_class, verbose=0)

print("loss : " + str(score[0]))
print("accuracy : " + str(score[1]))